# WMM Laboratorium 1

Autorzy:
- Kacper Siemionek, 331430
- Wojtek Zieziula, ebebebe

## Potrzebne biblioteki

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 1

## Zadanie 2

## Zadanie 3

## Zadanie 4